### OTA campaign to send MCCMAP list 250:02

In [1]:
import http.client
import xml.etree.ElementTree as ET
from os.path import join
import pandas as pd

def executeHTTP(request, url, path):
    client = http.client.HTTPConnection(url)
    client.request("POST", path, request, {"Content-Type": "text/xml"})
    resp = client.getresponse()
    return resp.read()

def sri4sm(ogt, msisdn):
    req = """<?xml version=\"1.0\"?>
        <ss7gw request=\"SRI_SM\">
        <d_ssn>6</d_ssn>
        <o_ssn>8</o_ssn>
        <o_gt>%s</o_gt>
        <d_gt>%s</d_gt>
        <msisdn>%s</msisdn>
        <priority>1</priority>
        <address>%s</address>
        </ss7gw>""" % (ogt, msisdn, msisdn, ogt)

    resp = executeHTTP(req, ss7_url, ss7_path)
    
    tree = ET.fromstring(resp)
    
    srifsm_dict = {'msisdn': msisdn,
                    'cgpa': ogt,
                    'result': '',
                    'imsi': '',
                    'msc': '',
                    'b_cgpa': '',
                    'error': '',
                    'content': resp}

    for child in tree.iter():
        if child.tag == 'result':
            srifsm_dict['result'] = child.text
        if child.tag == 'imsi':
            srifsm_dict['imsi'] = child.text
        if child.tag == 'msc':
            srifsm_dict['msc'] = child.text
        if child.tag == 'src_gt':
            srifsm_dict['b_cgpa'] = child.text
        if child.tag == 'error':
            srifsm_dict['error'] = child.text

    return srifsm_dict

def sendSMS(ogt, imsi, msc, payload):
    req = """<ss7gw request=\"MTSMS_OTA\">
        <d_ssn>8</d_ssn>
        <o_ssn>8</o_ssn>
        <o_gt>%s</o_gt>
        <d_gt>%s</d_gt>
        <imsi>%s</imsi>
        <password>123</password>
        <sc_number>%s</sc_number>
        <key_id>1</key_id>
        <command>1</command>
        <corr_id>1</corr_id>
        <seed>1122</seed>
        <msisdn>972559300036</msisdn>
        <message>%s</message>
        <sms_imsi>%s</sms_imsi>
        <map>3</map>
        </ss7gw>""" % (ogt, msc, imsi, ogt, payload, imsi)
    resp = executeHTTP(req, ss7_url, ss7_path)
    
    tree = ET.fromstring(resp)
    
    mtfsm_dict = {'result': '',
                 'b_cgpa': '',
                 'error': '',
                 'mccmnc': '',
                 'content': resp}

    for child in tree:
        if child.tag == 'error':
            mtfsm_dict['error'] = child.text
        if child.tag == 'result':
            mtfsm_dict['result'] = child.text
        if child.tag == 'src_gt':
            mtfsm_dict['b_cgpa'] = child.text
    
    return mtfsm_dict

def sendOTA(ogt, msisdn, payload):
    srifsm_dict = sri4sm(ogt, msisdn)

    if srifsm_dict['result'] == '0' and srifsm_dict['msc'] != cgpa:
        mtfsm_dict = sendSMS(ogt, srifsm_dict['imsi'], srifsm_dict['msc'], payload)
        return [srifsm_dict, mtfsm_dict]
    elif srifsm_dict['msc'] == cgpa:
        mtfsm_dict = {'result':'1','error':'Abroad'}
        return [srifsm_dict, mtfsm_dict]
    else:
        return [srifsm_dict]

def sendOTAcompaign(ogt, msisdn, mccmap_payload):
    resp = sendOTA(ogt, msisdn, mccmap_payload)

    if resp[0]['result'] == '0':
        mt_res = 'OK' if resp[1]['result'] == '0' else resp[1]['error']
        print(f"cgpa:{resp[0]['cgpa']};msisdn:{resp[0]['msisdn']};imsi:{resp[0]['imsi']};msc:{resp[0]['msc']};{mt_res}")
    else:
        print(f"cgpa:{resp[0]['cgpa']};msisdn:{resp[0]['msisdn']};{resp[0]['error']}")

USE [10028];
GO


SELECT
spn.PHONE_NUMBER
FROM SUBSCRIBERS s
INNER JOIN SUBSCRIBER_PHONE_NUMBERS spn ON s.subscriber_id = spn.subscriber_id
INNER JOIN ACCOUNTS a ON s.account_id = a.account_id
INNER JOIN SUBSCRIBER_STATUS ss ON s.SUBSCRIBER_ID = ss.SUBSCRIBER_ID
WHERE a.ACCOUNT_NAME = 'X2One test SIM-cards'
AND ss.END_DATE IS NULL
AND ss.status = 'Active'

In [2]:
ss7_url = "172.18.11.10"
ss7_path = "/cgi-bin/ss7gw.fcgi"

downloads = 'C:/Users/balob/Downloads'
msisdn_file = 'x2one_msisdn.csv'

mccmap_payload = "02700000270d00010000bfff0100000000000090dd53b30d1865a9c565c08aaa934e1e64ebb7816a808ecf42" # 250:02
cgpa = '972553316228'

df_msisdn = pd.read_csv(join(downloads, msisdn_file), header=None)
msisdn_list = df_msisdn[0][50:].to_list()
msisdn_list[:5]

[972000003077, 972000017640, 972000017650, 972000017725, 972000017741]

In [3]:
msisdn_list= [972000017634]
msisdn_list

[972000017634]

In [4]:
for msisdn in msisdn_list:
    sendOTAcompaign(cgpa, msisdn, mccmap_payload)

cgpa:972553316228;msisdn:972000017634;imsi:425120000012028;msc:972571209820;OK
